In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from math import log2

In [18]:
light_blue = Image.open('/Users/calvinatkins/Downloads/light_blue.jpeg')
orange = Image.open('/Users/calvinatkins/Downloads/orange.jpeg')
red = Image.open('/Users/calvinatkins/Downloads/red.jpeg')
turquoise = Image.open('/Users/calvinatkins/Downloads/turquoise.jpeg')
green = Image.open('/Users/calvinatkins/Downloads/green.jpeg')
test = Image.open('/Users/calvinatkins/Downloads/delete_later.webp')

In [19]:
def calculate_entropy(image):
    # Convert image to grayscale
    gray_image = image.convert('L')
    
    # Compute histogram
    histogram = gray_image.histogram()
    
    # Normalize histogram
    hist_normalized = np.array(histogram) / float(np.sum(histogram))
    
    # Calculate entropy
    entropy = -np.sum(hist_normalized * np.log2(hist_normalized + np.finfo(float).eps))
    
    return entropy
print(calculate_entropy(orange))
print(calculate_entropy(light_blue))
print(calculate_entropy(red))
print(calculate_entropy(turquoise))
print(calculate_entropy(green))

# if want value from 0-1 divide by max entropy value for RGB (8)

6.691576783046408
6.019709967973702
6.901819638579092
6.281817885853981
6.4738401361884215


In [20]:
# Testing to see if I can get similar values by other histograms I created
# can also test entropy of certain channels: red channel for ex
# grayscale is done to convert the image to a single channel for simplification

In [21]:
def calculate_entropy(channel):
    # Compute histogram
    histogram = channel.histogram()
    
    # Normalize histogram
    hist_normalized = np.array(histogram) / float(np.sum(histogram))
    
    # Calculate entropy
    entropy = -np.sum(hist_normalized * np.log2(hist_normalized + np.finfo(float).eps))
    
    return entropy

In [24]:
def calculate_channel_entropies(image):
    # Split image into RGB channels
    red_channel, green_channel, blue_channel = image.split()

    # Calculate entropy for each channel
    red_entropy = calculate_entropy(red_channel)
    green_entropy = calculate_entropy(green_channel)
    blue_entropy = calculate_entropy(blue_channel)

    print("Red Channel Entropy:", red_entropy)
    print("Green Channel Entropy:", green_entropy)
    print("Blue Channel Entropy:", blue_entropy)

print("Orange")
calculate_channel_entropies(orange)
print("Light Blue")
calculate_channel_entropies(light_blue)
print("Red")
calculate_channel_entropies(red)
print("Turquoise")
calculate_channel_entropies(turquoise)
print("Green")
calculate_channel_entropies(green)

Orange
Red Channel Entropy: 6.757419445124871
Green Channel Entropy: 6.916514807184205
Blue Channel Entropy: 6.650764129471522
Light Blue
Red Channel Entropy: 6.242466794361341
Green Channel Entropy: 6.048691287353764
Blue Channel Entropy: 6.352955993905189
Red
Red Channel Entropy: 6.265066472505103
Green Channel Entropy: 7.162446484540307
Blue Channel Entropy: 7.003592040802523
Turquoise
Red Channel Entropy: 6.9913544736995625
Green Channel Entropy: 6.782083637914582
Blue Channel Entropy: 6.778804986799511
Green
Red Channel Entropy: 7.024015974480801
Green Channel Entropy: 6.566294475401054
Blue Channel Entropy: 6.75972953385067
